In [41]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [43]:
tf.keras.backend.clear_session()

In [44]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [45]:
data = pd.read_csv('2003_mo_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego,Rio De Janeiro - Consumo de Cimento (t)
0,2003-1,0.291137,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,199.081280,0.773489,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,8.356512,254.362
1,2003-2,0.293538,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,198.561780,0.773547,3.587441e+08,1.814108e+07,22.149228,3.474056e+08,8.350332,252.842
2,2003-3,0.295914,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,198.773336,0.773606,3.591452e+08,1.814681e+07,22.154735,3.475266e+08,8.344152,229.385
3,2003-4,0.298264,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,198.753800,0.773665,3.595464e+08,1.815255e+07,22.160243,3.476476e+08,8.337971,243.407
4,2003-5,0.300588,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,198.721455,0.773724,3.599475e+08,1.815828e+07,22.165750,3.477686e+08,8.331791,256.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.559828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.917513,NaN,NaN,NaN,NaN,NaN,NaN,241.702
236,2022-9,0.558289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.658000,NaN,NaN,NaN,NaN,NaN,NaN,221.036
237,2022-10,0.556367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.339116,NaN,NaN,NaN,NaN,NaN,NaN,224.399
238,2022-11,0.554021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231.667333,NaN,NaN,NaN,NaN,NaN,NaN,221.319


In [46]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.438978,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.476880,0.839598,1.110414,-1.635096,-1.325228,-1.036232,1.324172
188,1.443990,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.460898,0.793634,1.098172,-1.625552,-1.326867,-1.053116,1.322796
189,1.457014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.421271,0.747671,1.085930,-1.616009,-1.328507,-1.070000,1.321421
190,1.470138,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.402090,0.701707,1.073688,-1.606465,-1.330147,-1.086883,1.320046


In [47]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 240, dtype: float64

In [48]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.395833,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-1.418673,1.438456,1.098176,-1.500012,-1.660268,-1.692180,1.473338
158,1.389960,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-1.470839,1.429097,1.105247,-1.521309,-1.646368,-1.657425,1.467585
159,1.384722,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-1.532363,1.419738,1.112318,-1.542605,-1.632469,-1.622671,1.461833
160,1.379646,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-1.608317,1.410379,1.119389,-1.563902,-1.618570,-1.587917,1.456081


In [49]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
157    175.917
158    208.629
159    181.933
160    187.947
161    181.556
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 162, dtype: float64

In [50]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [51]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [52]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
126,0.575814,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.995998,0.797097,0.862422,-0.029106,-0.391329,0.186886,0.731632
127,0.620950,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.926846,0.824296,0.875882,-0.083792,-0.444296,0.111052,0.766827
128,0.671632,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.836461,0.851494,0.889342,-0.138478,-0.497262,0.035217,0.802021
129,0.720432,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.767757,0.878692,0.902802,-0.193163,-0.550229,-0.040617,0.837216
130,0.766028,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.681375,0.905890,0.916261,-0.247849,-0.603195,-0.116452,0.872410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.438978,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.476880,0.839598,1.110414,-1.635096,-1.325228,-1.036232,1.324172
188,1.443990,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.460898,0.793634,1.098172,-1.625552,-1.326867,-1.053116,1.322796
189,1.457014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.421271,0.747671,1.085930,-1.616009,-1.328507,-1.070000,1.321421
190,1.470138,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.402090,0.701707,1.073688,-1.606465,-1.330147,-1.086883,1.320046


In [53]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [54]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [55]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')     
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [56]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [57]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3946607468, 3988040844, 1542492777, 2959799840, 3385896045, 986291796, 1071835165, 1901718809, 4128297327, 2236829336]


Step: 0 ___________________________________________
val_loss: 609.1598510742188
winner_seed: 3946607468


Step: 1 ___________________________________________
val_loss: 659.9403686523438


Step: 2 ___________________________________________
val_loss: 701.07177734375


Step: 3 ___________________________________________
val_loss: 646.59765625


Step: 4 ___________________________________________
val_loss: 576.740478515625
winner_seed: 3385896045


Step: 5 ___________________________________________
val_loss: 810.7489624023438


Step: 6 ___________________________________________
val_loss: 825.7866821289062


Step: 7 ___________________________________________
val_loss: 797.39111328125


Step: 8 ___________________________________________
val_loss: 705.674560546875


Step: 9 ___________________________________________
val_loss: 677.791748046875


final_seed: 3385896045

In [58]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 111ms/step - loss: 125286.4922 - val_loss: 123110.9453
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 124730.8125 - val_loss: 122494.0547
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 124087.8438 - val_loss: 121822.0938
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 123407.5391 - val_loss: 121121.4141
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 122724.4844 - val_loss: 120459.5156
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 122063.7812 - val_loss: 119860.1406
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 121464.2656 - val_loss: 119310.3438
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 120944.6875 - val_loss: 118819.0859
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 120470.3438 - val_loss: 118411.6406
Epoch 10/10000
4/

4/4 [==============================] - 0s 7ms/step - loss: 112141.0547 - val_loss: 110439.0312
Epoch 76/10000
4/4 [==============================] - 0s 7ms/step - loss: 112041.9297 - val_loss: 110339.8906
Epoch 77/10000
4/4 [==============================] - 0s 7ms/step - loss: 111941.8984 - val_loss: 110241.2891
Epoch 78/10000
4/4 [==============================] - 0s 7ms/step - loss: 111841.5703 - val_loss: 110142.9688
Epoch 79/10000
4/4 [==============================] - 0s 7ms/step - loss: 111742.1328 - val_loss: 110044.9453
Epoch 80/10000
4/4 [==============================] - 0s 7ms/step - loss: 111643.8906 - val_loss: 109946.8594
Epoch 81/10000
4/4 [==============================] - 0s 7ms/step - loss: 111545.4141 - val_loss: 109849.4922
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 111446.7344 - val_loss: 109751.7188
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 111346.5469 - val_loss: 109654.4062
Epoch 84/10000
4/4 [=====

4/4 [==============================] - 0s 7ms/step - loss: 105002.1484 - val_loss: 103383.0391
Epoch 150/10000
4/4 [==============================] - 0s 7ms/step - loss: 104910.2422 - val_loss: 103290.7891
Epoch 151/10000
4/4 [==============================] - 0s 7ms/step - loss: 104816.1953 - val_loss: 103198.8516
Epoch 152/10000
4/4 [==============================] - 0s 7ms/step - loss: 104722.7969 - val_loss: 103106.7812
Epoch 153/10000
4/4 [==============================] - 0s 7ms/step - loss: 104630.8359 - val_loss: 103014.9141
Epoch 154/10000
4/4 [==============================] - 0s 7ms/step - loss: 104537.9062 - val_loss: 102923.4375
Epoch 155/10000
4/4 [==============================] - 0s 7ms/step - loss: 104444.5547 - val_loss: 102832.4219
Epoch 156/10000
4/4 [==============================] - 0s 7ms/step - loss: 104352.7031 - val_loss: 102741.0938
Epoch 157/10000
4/4 [==============================] - 0s 7ms/step - loss: 104261.1797 - val_loss: 102649.7656
Epoch 158/10000
4

4/4 [==============================] - 0s 7ms/step - loss: 98293.4922 - val_loss: 96752.4688
Epoch 224/10000
4/4 [==============================] - 0s 7ms/step - loss: 98205.3438 - val_loss: 96665.2812
Epoch 225/10000
4/4 [==============================] - 0s 7ms/step - loss: 98117.2500 - val_loss: 96578.3438
Epoch 226/10000
4/4 [==============================] - 0s 7ms/step - loss: 98029.1875 - val_loss: 96491.4844
Epoch 227/10000
4/4 [==============================] - 0s 7ms/step - loss: 97941.4297 - val_loss: 96404.4688
Epoch 228/10000
4/4 [==============================] - 0s 7ms/step - loss: 97853.2891 - val_loss: 96317.6250
Epoch 229/10000
4/4 [==============================] - 0s 7ms/step - loss: 97765.6953 - val_loss: 96230.7500
Epoch 230/10000
4/4 [==============================] - 0s 7ms/step - loss: 97677.2422 - val_loss: 96144.0234
Epoch 231/10000
4/4 [==============================] - 0s 7ms/step - loss: 97590.5156 - val_loss: 96056.9219
Epoch 232/10000
4/4 [==============

4/4 [==============================] - 0s 7ms/step - loss: 91846.5703 - val_loss: 90382.1797
Epoch 299/10000
4/4 [==============================] - 0s 7ms/step - loss: 91762.3203 - val_loss: 90299.5469
Epoch 300/10000
4/4 [==============================] - 0s 7ms/step - loss: 91679.2031 - val_loss: 90216.8984
Epoch 301/10000
4/4 [==============================] - 0s 7ms/step - loss: 91595.1875 - val_loss: 90134.9297
Epoch 302/10000
4/4 [==============================] - 0s 7ms/step - loss: 91512.6094 - val_loss: 90052.9688
Epoch 303/10000
4/4 [==============================] - 0s 7ms/step - loss: 91429.5859 - val_loss: 89971.2109
Epoch 304/10000
4/4 [==============================] - 0s 7ms/step - loss: 91346.3672 - val_loss: 89889.4688
Epoch 305/10000
4/4 [==============================] - 0s 7ms/step - loss: 91263.5156 - val_loss: 89807.3438
Epoch 306/10000
4/4 [==============================] - 0s 7ms/step - loss: 91180.5469 - val_loss: 89724.8594
Epoch 307/10000
4/4 [==============

...

Epoch 4608/10000
4/4 [==============================] - 0s 7ms/step - loss: 209.1723 - val_loss: 1023.4704
Epoch 4609/10000
4/4 [==============================] - 0s 7ms/step - loss: 196.3978 - val_loss: 1028.4156
Epoch 4610/10000
4/4 [==============================] - 0s 7ms/step - loss: 207.4760 - val_loss: 1033.5144
Epoch 4611/10000
4/4 [==============================] - 0s 7ms/step - loss: 211.2209 - val_loss: 1012.9099
Epoch 4612/10000
4/4 [==============================] - 0s 7ms/step - loss: 197.0999 - val_loss: 1004.6304
Epoch 4613/10000
4/4 [==============================] - 0s 7ms/step - loss: 205.2700 - val_loss: 992.2347
Epoch 4614/10000
4/4 [==============================] - 0s 7ms/step - loss: 203.0522 - val_loss: 968.0420
Epoch 4615/10000
4/4 [==============================] - 0s 7ms/step - loss: 216.5443 - val_loss: 972.6140
Epoch 4616/10000
4/4 [==============================] - 0s 7ms/step - loss: 209.3799 - val_loss: 968.7700
Epoch 4617/10000
4/4 [===================

4/4 [==============================] - 0s 7ms/step - loss: 204.0934 - val_loss: 952.2744
Epoch 4686/10000
4/4 [==============================] - 0s 7ms/step - loss: 198.7258 - val_loss: 940.1987
Epoch 4687/10000
4/4 [==============================] - 0s 7ms/step - loss: 216.7925 - val_loss: 956.8314
Epoch 4688/10000
4/4 [==============================] - 0s 7ms/step - loss: 198.3726 - val_loss: 967.2758
Epoch 4689/10000
4/4 [==============================] - 0s 7ms/step - loss: 198.1772 - val_loss: 960.1448
Epoch 4690/10000
4/4 [==============================] - 0s 7ms/step - loss: 206.2795 - val_loss: 956.1292
Epoch 4691/10000
4/4 [==============================] - 0s 7ms/step - loss: 200.2418 - val_loss: 980.0202
Epoch 4692/10000
4/4 [==============================] - 0s 7ms/step - loss: 199.5459 - val_loss: 1002.1155
Epoch 4693/10000
4/4 [==============================] - 0s 7ms/step - loss: 210.8656 - val_loss: 964.9597
Epoch 4694/10000
4/4 [==============================] - 0s 7ms

4/4 [==============================] - 0s 7ms/step - loss: 199.7094 - val_loss: 919.0184
Epoch 4763/10000
4/4 [==============================] - 0s 7ms/step - loss: 180.4073 - val_loss: 922.4136
Epoch 4764/10000
4/4 [==============================] - 0s 7ms/step - loss: 198.2350 - val_loss: 927.0088
Epoch 4765/10000
4/4 [==============================] - 0s 7ms/step - loss: 172.5649 - val_loss: 926.8145
Epoch 4766/10000
4/4 [==============================] - 0s 7ms/step - loss: 204.9614 - val_loss: 941.4518
Epoch 4767/10000
4/4 [==============================] - 0s 7ms/step - loss: 193.6023 - val_loss: 954.8266
Epoch 4768/10000
4/4 [==============================] - 0s 7ms/step - loss: 207.6075 - val_loss: 941.1656
Epoch 4769/10000
4/4 [==============================] - 0s 7ms/step - loss: 188.5764 - val_loss: 1132.1499
Epoch 4770/10000
4/4 [==============================] - 0s 7ms/step - loss: 280.2131 - val_loss: 1139.7914
Epoch 4771/10000
4/4 [==============================] - 0s 7m

Epoch 4839/10000
4/4 [==============================] - 0s 7ms/step - loss: 190.0868 - val_loss: 1036.5559
Epoch 4840/10000
4/4 [==============================] - 0s 7ms/step - loss: 212.4615 - val_loss: 1043.3044
Epoch 4841/10000
4/4 [==============================] - 0s 7ms/step - loss: 198.0619 - val_loss: 922.2566
Epoch 4842/10000
4/4 [==============================] - 0s 7ms/step - loss: 222.8177 - val_loss: 958.4529
Epoch 4843/10000
4/4 [==============================] - 0s 7ms/step - loss: 212.2441 - val_loss: 1062.4622
Epoch 4844/10000
4/4 [==============================] - 0s 7ms/step - loss: 212.7025 - val_loss: 979.2203
Epoch 4845/10000
4/4 [==============================] - 0s 7ms/step - loss: 192.8428 - val_loss: 910.2316
Epoch 4846/10000
4/4 [==============================] - 0s 7ms/step - loss: 218.5971 - val_loss: 907.6158
Epoch 4847/10000
4/4 [==============================] - 0s 7ms/step - loss: 205.7540 - val_loss: 951.2849
Epoch 4848/10000
4/4 [=====================

4/4 [==============================] - 0s 7ms/step - loss: 208.9951 - val_loss: 972.9820
Epoch 4917/10000
4/4 [==============================] - 0s 7ms/step - loss: 221.7528 - val_loss: 940.2866
Epoch 4918/10000
4/4 [==============================] - 0s 7ms/step - loss: 211.2318 - val_loss: 993.7426
Epoch 4919/10000
4/4 [==============================] - 0s 7ms/step - loss: 231.1636 - val_loss: 1030.4875
Epoch 4920/10000
4/4 [==============================] - 0s 7ms/step - loss: 228.2624 - val_loss: 1015.0688
Epoch 4921/10000
4/4 [==============================] - 0s 7ms/step - loss: 211.0996 - val_loss: 998.2122
Epoch 4922/10000
4/4 [==============================] - 0s 7ms/step - loss: 195.2322 - val_loss: 999.5892
Epoch 4923/10000
4/4 [==============================] - 0s 7ms/step - loss: 201.3423 - val_loss: 994.8018
Epoch 4924/10000
4/4 [==============================] - 0s 7ms/step - loss: 199.2565 - val_loss: 1003.1448
Epoch 4925/10000
4/4 [==============================] - 0s 7

In [59]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [60]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,189.153839,189.171799,189.147476,189.151443,189.140717,189.130295,189.147873,189.192932,189.177292,189.160172,189.143524,189.123184,189.04541,189.053833,189.028305,188.984543,188.950226,188.906357,188.909744,188.889999,188.866928,188.827042,188.800705,188.764542,188.736435,188.691208,188.62468,188.564743,188.512024,188.471115
Target,185.846,200.671,207.913,188.107,193.661,175.783,185.078,146.761,173.121,169.841,142.706,242.422,226.205,229.696,232.624,234.262,217.727,190.769,232.609,195.364,205.69,231.354,236.711,215.36,258.015,244.87,225.41,252.849,226.618,190.993
Error,3.307846,11.499207,18.765518,1.044449,4.520279,13.34729,4.06987,42.431931,16.05629,19.319168,46.437531,53.298813,37.159592,40.642166,43.595688,45.277451,28.776779,1.86264,43.699249,6.473999,16.823074,42.526962,47.910294,26.595459,69.27858,56.178787,36.785324,64.284256,38.105972,2.521881


In [61]:
display(mae)
display(mape)

29.419878

0.13687028

In [62]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [63]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2269.8406]] - Target[2211.91]| =  Error: [[57.930664]]; MAPE:[[0.02619034]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2267.0278]] - Target[2648.3710000000005]| =  Error: [[381.34326]]; MAPE:[[0.14399162]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[1131.6002]] - Target[1398.7549999999999]| =  Error: [[267.1548]]; MAPE:[[0.1909947]]


[array([[57.930664]], dtype=float32),
 array([[381.34326]], dtype=float32),
 array([[267.1548]], dtype=float32)]

235.47624

0.120392226